In [1]:
from keras.layers.core import Activation, Dense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import nltk  #用来分词
import collections  #用来统计词频
import numpy as np

Using TensorFlow backend.


In [2]:
import codecs
maxlen = 0  #句子最大长度
word_freqs = collections.Counter()  #词频
num_recs = 0 # 样本数
with codecs.open('E:/old collect tweet/train.txt','r+','utf-8') as f:
    for line in f:
        label, sentence = line.strip().split("\t")
        words = nltk.word_tokenize(sentence.lower())
        if len(words) > maxlen:
            maxlen = len(words)
        for word in words:
            word_freqs[word] += 1
        num_recs += 1
print('max_len ',maxlen)
print('nb_words ', len(word_freqs))

max_len  32
nb_words  24490


In [3]:
i = 0
for a in word_freqs.values():
    if a > 2:
        i = i + 1
print(i)

6748


In [4]:
MAX_FEATURES = 6750
MAX_SENTENCE_LENGTH = 30

In [5]:
vocab_size = min(MAX_FEATURES, len(word_freqs)) + 2
word2index = {x[0]: i+2 for i, x in enumerate(word_freqs.most_common(MAX_FEATURES))}
word2index["PAD"] = 0
word2index["UNK"] = 1
index2word = {v:k for k, v in word2index.items()}

In [6]:
X = np.empty(num_recs,dtype=list)
y = np.zeros(num_recs)
i=0
with open('E:/old collect tweet/train.txt','r+') as f:
    for line in f:
        label, sentence = line.strip().split("\t")
        words = nltk.word_tokenize(sentence.lower())
        seqs = []
        for word in words:
            if word in word2index:
                seqs.append(word2index[word])
            else:
                seqs.append(word2index["UNK"])
        X[i] = seqs
        y[i] = int(label)
        i += 1
X = sequence.pad_sequences(X, maxlen=MAX_SENTENCE_LENGTH)

In [7]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
EMBEDDING_SIZE = 128
HIDDEN_LAYER_SIZE = 64
BATCH_SIZE = 32
NUM_EPOCHS = 10

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE,input_length=MAX_SENTENCE_LENGTH))
model.add(LSTM(HIDDEN_LAYER_SIZE, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,validation_data=(Xtest, ytest))

Train on 10668 samples, validate on 2668 samples
Epoch 1/10
10668/10668 [==============================] - 21s - loss: 0.2110 - acc: 0.9417 - val_loss: 0.1217 - val_acc: 0.9636
Epoch 2/10
10668/10668 [==============================] - 17s - loss: 0.0808 - acc: 0.9753 - val_loss: 0.1130 - val_acc: 0.9644
Epoch 3/10
10668/10668 [==============================] - 18s - loss: 0.0461 - acc: 0.9859 - val_loss: 0.1101 - val_acc: 0.9719
Epoch 4/10
10668/10668 [==============================] - 18s - loss: 0.0275 - acc: 0.9921 - val_loss: 0.1697 - val_acc: 0.9618
Epoch 5/10
10668/10668 [==============================] - 18s - loss: 0.0203 - acc: 0.9941 - val_loss: 0.1596 - val_acc: 0.9610
Epoch 6/10
10668/10668 [==============================] - 18s - loss: 0.0149 - acc: 0.9953 - val_loss: 0.1912 - val_acc: 0.9659
Epoch 7/10
10668/10668 [==============================] - 17s - loss: 0.0113 - acc: 0.9967 - val_loss: 0.1952 - val_acc: 0.9588
Epoch 8/10
10668/10668 [==============================]

In [9]:
score, acc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)
print("\nTest score: %.3f, accuracy: %.3f" % (score, acc))
print('{}   {}      {}'.format('predict label','label','text'))
for i in range(10):
    idx = np.random.randint(len(Xtest))
    xtest = Xtest[idx].reshape(1,30)
    ylabel = ytest[idx]
    ypred = model.predict(xtest)[0][0]
    sent = " ".join([index2word[x] for x in xtest[0] if x != 0])
    print('      {}           {}     {}'.format(int(round(ypred)), int(ylabel), sent))

2624/2668 [============================>.] - ETA: 0s
Test score: 0.227, accuracy: 0.957
predict label   label      text
      0           0     rt stop trying to look perfect the most flawless bananas usually taste gross
      0           0     rt my heart is heavy rip chester
      0           0     an UNK bro version of UNK love this glass it 's so heavy feel like jr UNK at the oil
      0           0     life 's like head full of amazing thick hair people trying to count your whole head as or UNK like really split some UNK
      0           0     # nigeria # rain # UNK # heavyrain # kid # dancing love the rain love the dancing ps rains
      1           1     did someone say breakfast pick up bag of our mouth-watering thick cut UNK bacon express
      0           0     helped nearby drivers by reporting heavy traffic jam on manor pl edinburgh on drive social
      0           0     acr 125 engross UNK need awareness week
      0           0     rt with the help of to reduce food was

In [10]:
# import json
# oldtweet = open('E:/middleTwitter.json','r',encoding="utf8")
# i = 0
# alltext = []
# for line in oldtweet:
#     try:
#         new = json.loads(line,encoding = 'utf8')
#         alltext.append(new['text'])
#         i = i + 1
#         if i % 10000 == 30:
#             print(i)
#     except BaseException as e:
#         print('Error on_data:'+str(e)) 
# oldtweet.close()
# print(len(alltext))
import json
oldtweet = open('E:/old collect tweet/hysimpcollected.json','r',encoding="utf8")
i = 0
alltext = []
for line in oldtweet:
    try:
        new = json.loads(line,encoding = 'utf8')
        alltext.append(new['text'])
        i = i + 1
        if i % 10000 == 30:
            print(i)
    except BaseException as e:
        print('Error on_data:'+str(e)) 
oldtweet.close()
print(len(alltext))

30
10030
20030
30030
40030
50030
60030
70030
80030
90030
100030
110030
120030
130030
140030
150030
153984


In [11]:
import re
from nltk.corpus import stopwords
import string
stopwords = set(stopwords.words('english'))
delpunc = string.punctuation
    
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
# print(preprocess(tweet))
def rmhttp(list_str):
    rmhttp = []
    for a in list_str:
        if 'http://' in a:
            continue
        elif 'https://' in a:
            continue
        elif '@' in a:
            continue
        elif len(a) == 1:
            if ord(a) < 32 or ord(a) > 126:
                continue
#         elif a.lower() in stopwords:
#             continue
#         elif a in delpunc:
#             continue        
        else:
            rmhttp.append(a)
    return ' '.join(rmhttp)
tweettest = u'RT @marcobonzanini: just an example! :D http://example.com #NLP'

In [12]:
cleantext = []
for a in alltext:
    cleantext.append(rmhttp(preprocess(a)))
print(len(cleantext))

153984


In [13]:
# INPUT_SENTENCES = ['I love reading.','You are so boring.']
INPUT_SENTENCES = cleantext
XX = np.empty(len(INPUT_SENTENCES),dtype=list)
i=0
for sentence in  INPUT_SENTENCES:
    words = nltk.word_tokenize(sentence.lower())
    seq = []
    for word in words:
        if word in word2index:
            seq.append(word2index[word])
        else:
            seq.append(word2index['UNK'])
    XX[i] = seq
    i+=1

XX = sequence.pad_sequences(XX, maxlen=MAX_SENTENCE_LENGTH)
labels = [int(round(x[0])) for x in model.predict(XX) ]
label2word = {1:'fat fastfood', 0:'not'}
a = 0
for i in range(len(INPUT_SENTENCES)):
    print('{}   {}'.format(label2word[labels[i]], INPUT_SENTENCES[i]))
    a = a + 1
    if a > 10:
        break

not   Couldn't have got through that slab without this bad boy from our mates haleyhire I'm
not   It's the th of April had seriously bad dream last night killed spider the night before in my dream spiders
not   Oh My Yoga have my very own yoga page on Instagram FOLLOW omy_australia if you'd like
not   am coffee fanatic Once you go to proper coffee you can't go back You cannot go back
not   Bought jacket City Chic in Docklands VIC
not   Written in style John Clarke would appreciate
not   bigbosbbq No chance was sacrificing this hot meal for some fancy photos tonight Double
not   Working in an industry where got to meet lot of tourist from
not   #best Apr 10 #melbourne #australia #travel #money offers app guides you to #cheap #forex
not   #Repost EHPlabs ambassador and my fellow My favorite post workout supplement
not   Beer was made by men wine by God Yarra Valley Wineries Tour


In [14]:
fastnum = 0
allnum = 0
for i in range(len(INPUT_SENTENCES)):
    allnum = allnum + 1
    if len(label2word[labels[i]]) == 12:
        fastnum = fastnum + 1
print(fastnum)
print(allnum)
print(label2word[labels[0]])

18775
153984
not


In [19]:
import json
import codecs
firstfile = open('E:/old collect tweet/hysimpcollected.json','r',encoding="utf8")
secondfile = codecs.open('E:/old collect tweet/hylabelledTwitter.json','w',encoding='utf-8')
alltext = []
i = 0
for line in firstfile:
    text = []
    try:
        tweet = json.loads(line)
        obj = {"created_at":tweet['created_at'],"label":label2word[labels[i]],"text":tweet['text'],"id":tweet['id'],"coordinate":tweet['coordinates']}
#             print obj
        aaaa = json.dumps(obj, ensure_ascii=False)
        secondfile.write(aaaa+'\n')
        i = i+1
        if i % 10000 == 30:
            print(i)
    except BaseException as e:
        print('Error on_data:'+str(e))
print('complete')
firstfile.close()
secondfile.close()

30
10030
20030
30030
40030
50030
60030
70030
80030
90030
100030
110030
120030
130030
140030
150030
complete
